In [1]:
import scipy
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
df = pd.read_csv("Data.csv", sep=";")
df.dtypes

Direksi Perempuan    int64
Direksi Laki-Laki    int64
Harga penutupan      int64
Total Shares         int64
Total Utang          int64
Total Asset          int64
dtype: object

In [3]:
#X = (df['Direksi Perempuan'])/(df['Direksi Perempuan'] + df['Direksi Laki-Laki'])
X = df["Direksi Perempuan"]
nilai_pasar = df["Total Shares"] * df["Harga penutupan"]
np_plus_liability = nilai_pasar + df["Total Utang"]
#Y = np_plus_liability / df["Total Asset"]
Y = nilai_pasar / df["Total Asset"]

data = pd.DataFrame({"X":X, "Y":Y})
X_mean = np.mean(data["X"])
X_std = np.std(data["X"])
Y_mean = np.mean(data["Y"])
Y_std = np.std(data["Y"])

In [4]:
data_sample = pd.DataFrame({"X": X, "Y": Y})
data_sample.describe()

,X,Y
count,140.000000,140.000000
mean,1.300000,0.540043
std,1.239053,2.141017
min,0.000000,0.007332
25%,0.000000,0.096924
50%,1.000000,0.140086
75%,2.000000,0.277923
max,6.000000,18.006205


In [5]:
# data_sample.drop(data_sample[data_sample["X"] == 0].index, inplace=True)
# np.size(np.where(data_sample["X"] == 0))

In [6]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.6881
Date:                Sun, 07 May 2023   Prob (F-statistic):              0.408
Time:                        23:41:07   Log-Likelihood:                -304.38
No. Observations:                 140   AIC:                             612.8
Df Residuals:                     138   BIC:                             618.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6983      0.263      2.654      0.009       0.178       1.218
X             -0.1217      0.147     -0.830      0.408      -0.412       0.168
==============================================================================
Omnibus:                      238.216   Durbin-Watson:                   0.880
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22048.223
Skew:                           7.694   Prob(JB):                         0.00
Kurtosis:                      62.522   Cond. No.                         3.09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
bX = data_sample["X"] * result.params[0]
abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [8]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.162
Model:                            OLS   Adj. R-squared:                  0.156
Method:                 Least Squares   F-statistic:                     26.63
Date:                Sun, 07 May 2023   Prob (F-statistic):           8.41e-07
Time:                        23:41:08   Log-Likelihood:                -282.53
No. Observations:                 140   AIC:                             569.1
Df Residuals:                     138   BIC:                             574.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8120      0.225      3.608      0.000       0.367       1.257
X              0.6477      0.126      5.160      0.000       0.400       0.896
==============================================================================
Omnibus:                      244.279   Durbin-Watson:                   1.007
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            24904.080
Skew:                           8.029   Prob(JB):                         0.00
Kurtosis:                      66.336   Cond. No.                         3.09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
XQ3,XQ1 = np.percentile(data_sample["X"], [75 ,25])
XIQR = XQ3 - XQ1 

In [10]:
lower_X = XQ1 - (1.5 * XIQR)
upper_X = XQ3 + (1.5 * XIQR)

In [11]:
data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)]

70    6
71    6
72    6
Name: X, dtype: int64

In [12]:
data_sample.drop(data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)].index, inplace=True)

In [13]:
YQ3,YQ1 = np.percentile(data_sample['Y'], [75 ,25])
YIQR = YQ3 - YQ1 

In [14]:
lower_Y = YQ1 - (1.5 * YIQR)
upper_Y = YQ3 + (1.5 * YIQR)

In [15]:
data_sample.drop(data_sample["Y"][(data_sample["Y"] < lower_Y) | (data_sample["Y"] > upper_Y)].index, inplace=True)

In [16]:
data_sample.describe()

,X,Y
count,120.000000,120.000000
mean,1.308333,0.159487
std,1.035470,0.100648
min,0.000000,0.007332
25%,1.000000,0.092320
50%,1.000000,0.130958
75%,2.000000,0.202564
max,4.000000,0.456443


In [17]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     3.760
Date:                Sun, 07 May 2023   Prob (F-statistic):             0.0549
Time:                        23:41:08   Log-Likelihood:                 107.65
No. Observations:                 120   AIC:                            -211.3
Df Residuals:                     118   BIC:                            -205.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1371      0.015      9.346      0.000       0.108       0.166
X              0.0171      0.009      1.939      0.055      -0.000       0.035
==============================================================================
Omnibus:                       21.153   Durbin-Watson:                   1.080
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               26.421
Skew:                           1.130   Prob(JB):                     1.83e-06
Kurtosis:                       3.419   Cond. No.                         3.36
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
bX = data_sample["X"] * result.params["X"]

In [19]:
# Y = a + bX + e
# e = Y - a - bX

abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [20]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.533
Date:                Sun, 07 May 2023   Prob (F-statistic):              0.218
Time:                        23:41:08   Log-Likelihood:                 161.71
No. Observations:                 120   AIC:                            -319.4
Df Residuals:                     118   BIC:                            -313.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0666      0.009      7.122      0.000       0.048       0.085
X              0.0070      0.006      1.238      0.218      -0.004       0.018
==============================================================================
Omnibus:                       24.973   Durbin-Watson:                   1.300
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               32.475
Skew:                           1.198   Prob(JB):                     8.87e-08
Kurtosis:                       3.871   Cond. No.                         3.36
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""